In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import comb
import random

In [2]:
def delta (theta, a, b, c):
    return ((1+b)/((c**(theta))*((a**(-theta))+b)))

### Forward
- u,d に一様乱数で誤差をつける.(-0.2<ε<0.2)
- probも変化させるか

In [12]:
def forward(N, theta, u, d, p, default_x):

    random.seed(123)

    q = (1-d)/(u-d)
    a = ((1-p)/p)*(q/(1-q))
    b = (1-q)/q
    c = (1-p)/(1-q)

    stock = []
    times = []
    up = []
    down = []
    
    prob = []
    
    for i in range(N+1):
        stock.append([])
        up.append([])
        down.append([])
        times.append(i)
        for j in range(0,N+1):
            stock[i].append([])
        for k in range(i):
            up[i].append(u+(random.random()*(-(2/5))+0.2))
            down[i].append(d+(random.random()*(-(2/5))+0.2))
    for i in range(N+1):
        for j in range(N+1):
            stock[i][0] = i
            stock[i][1] = times[0:i+1]
            stock[i][2] = times[i::-1]
            stock[i][3] = times[0:i+1]
            stock[i][4] = theta
            stock[i][5] = up[i]
            stock[i][6] = down[i]
            stock[i][7] = p
            stock[i][8] = list(map(lambda x: 0 if x==0 else x, times[0:i+1]))
            stock[i][9] = list(map(lambda x: 0 if x==0 else x, times[i::-1]))
            #price
            #stock[i][10] = list(map(lambda x, y: default_x if x==0 and y==0 else (default_x*x)+(default_x*y) if x==0 or y==0 else (default_x*x)*(default_x*y), stock[i][8], stock[i][9]))
            stock[i][10] = list(map(lambda k, l, m, n: default_x if m==0 and n==0 else (m)+(n), stock[i][5], stock[i][6], stock[i][8], stock[i][9]))
            #U_0
            stock[i][11] = list(map(lambda x: ((1-(1/theta))**(-1))*(x**(1-(1/theta))), stock[i][10]))
            #I_0
            #stock[i][10] = list(map(lambda x: (x**(-1/theta))**(-1), stock[i][8]))
            #I_1
            #stock[i][11] = list(map(lambda x: (1+b)/((c**theta)*((a**(-theta))+b)*x, stock[i][10]))
            #U_1
            stock[i][12] = list(map(lambda x: (((1+b)/((c**(theta))*((a**(-theta))+b)))**(1/theta))*x, stock[i][11]))
            #pi
            stock[i][13] = list(map(lambda x: ((((1+b)/((c**theta)*((a**(-theta))+b)))*((p/q)**theta)-1)/(u-1))*x, stock[i][10]))
            #Exp[U_1]
            stock[i][14] = list(map(lambda  l, m, n : comb(i, l, exact=True)*(p**l)*((1-p)**m)*n if (l!=0 and m!=0) \
                                 else (p**l)*n if m==0 \
                                 else ((1-p)**m)*n if l==0 \
                                 else nan \
                                 , stock[i][2], stock[i][3], stock[i][12]))
    df = pd.DataFrame(stock)        
    df = df.set_index(0)
    df.columns = ["node_num", "up_times", "down_times", "theta", "up", "down", "prob", "up_r", "down_r", "price", "U_0", "U_1", "alpha", "Exp_U1"]

    last_u1_mean = list(map(lambda x: np.mean(x), df["Exp_U1"]))[14]
    
    return df
    

In [15]:
df =forward(14, 3, 1.2, 0.8, 0.4, 100)

In [16]:
df.head()

,node_num,up_times,down_times,theta,up,down,prob,up_r,down_r,price,U_0,U_1,alpha,Exp_U1
0,,,,,,,,,,,,,,
0,[0],[0],[0],3,[],[],0.4,[0],[0],[100],[32.316520350478264],[31.118495001234137],[-271.4285714285714],[31.118495001234137]
1,"[0, 1]","[1, 0]","[0, 1]",3,[1.3790545604596223],[0.9651253289909472],0.4,"[0, 1]","[1, 0]","[0, 0]","[0.0, 0.0]","[0.0, 0.0]","[-0.0, -0.0]","[0.0, 0.0]"
2,"[0, 1, 2]","[2, 1, 0]","[0, 1, 2]",3,"[1.2371032945318405, 1.0395204488193222]","[0.9569199060246244, 0.984738533559071]",0.4,"[0, 1, 2]","[2, 1, 0]","[0, 1, 0]","[0.0, 1.4999999999999998, 0.0]","[0.0, 1.444392589165634, 0.0]","[-0.0, -2.714285714285714, -0.0]","[0.0, 0.6933084427995043, 0.0]"
3,"[0, 1, 2, 3]","[3, 2, 1, 0]","[0, 1, 2, 3]",3,"[1.1855191839864292, 1.0591653524282525, 1.265...","[0.8671209205961281, 0.9361350413112121, 0.866...",0.4,"[0, 1, 2, 3]","[3, 2, 1, 0]","[0, 2, 2, 0]","[0.0, 2.381101577952299, 2.381101577952299, 0.0]","[0.0, 2.292830315496599, 2.292830315496599, 0.0]","[-0.0, -5.428571428571428, -5.428571428571428,...","[0.0, 0.6603351308630205, 0.9905026962945309, ..."
4,"[0, 1, 2, 3, 4]","[4, 3, 2, 1, 0]","[0, 1, 2, 3, 4]",3,"[1.3019346589929555, 1.2254896826339126, 1.160...","[0.9993317785683109, 0.9649546000998309, 0.972...",0.4,"[0, 1, 2, 3, 4]","[4, 3, 2, 1, 0]","[0, 3, 4, 3, 0]","[0.0, 3.1201257345778557, 3.779763149684619, 3...","[0.0, 3.00445765885949, 3.6396412548038795, 3....","[-0.0, -8.142857142857142, -10.857142857142856...","[0.0, 0.4614846964008178, 1.257860017660221, 1..."


In [48]:
default_x=100
for i in range(4):
    print([list(map(lambda x, y, z: default_x if x==0 and y==0 else default_x*z, df.up_r[i], df.down_r[i], df.up[i])), \
          list(map(lambda x, y, z: default_x if x==0 and y==0 else default_x*z, df.up_r[i], df.down_r[i], df.down[i]))])

[[], []]
[[137.90545604596224], [96.51253289909472]]
[[123.71032945318406, 103.95204488193221], [95.69199060246244, 98.47385335590711]]
[[118.55191839864293, 105.91653524282525, 126.51133371562896], [86.71209205961281, 93.6135041311212, 86.64814421484179]]


IndexError: list index out of range

In [71]:
tmp =[]
for i in range(1, 1000):
    tmp.append([])
    tmp.append(random.random()*(-(2/5))+0.2)

In [59]:
tmp = []
u=1.2
random.seed(123)
for i in range(len(df)):
    tmp.append([])    
    for j in range(i):
        tmp[i].append(u+random.random()*(-(2/5))+0.2)        

In [152]:
random.seed(123)

u=1.2
d=0.8
p=0.4
N=3

q = (1-d)/(u-d)
a = ((1-p)/p)*(q/(1-q))
b = (1-q)/q
c = (1-p)/(1-q)

x=100
    
stock = []
tmp = []
up = []
down = []
prob = []
    
for i in range(N+1):
    stock.append([])
    up.append([])
    down.append([])
    tmp.append(i)
    for j in range(0,N+1):
        stock[i].append([])
    for k in range(i):
        up[i].append(u+random.random()*(-(2/5))+0.2)
        down[i].append(d+random.random()*(-(2/5))+0.2)
    

In [170]:
tmp=[]
for i in range(len(up)+1):
    tmp[i]=list(map(lambda x: x*100, up[i]))

IndexError: list assignment index out of range

[118.55191839864293, 105.91653524282525, 126.51133371562896]

[1.3790545604596223]